In [5]:
from fileinput import filename

import polars as pl
import polars.selectors as cs
import time
import chardet

from numpy.f2py.cfuncs import includes
from sqlalchemy.dialects.mssql.information_schema import columns

from notebooks.chapter_05 import trips

penguins = pl.read_csv("../data/penguins.csv")
print(penguins)

shape: (344, 9)
┌───────┬───────────┬───────────┬────────────────┬───┬───────────────┬─────────────┬────────┬──────┐
│ rowid ┆ species   ┆ island    ┆ bill_length_mm ┆ … ┆ flipper_lengt ┆ body_mass_g ┆ sex    ┆ year │
│ ---   ┆ ---       ┆ ---       ┆ ---            ┆   ┆ h_mm          ┆ ---         ┆ ---    ┆ ---  │
│ i64   ┆ str       ┆ str       ┆ str            ┆   ┆ ---           ┆ str         ┆ str    ┆ i64  │
│       ┆           ┆           ┆                ┆   ┆ str           ┆             ┆        ┆      │
╞═══════╪═══════════╪═══════════╪════════════════╪═══╪═══════════════╪═════════════╪════════╪══════╡
│ 1     ┆ Adelie    ┆ Torgersen ┆ 39.1           ┆ … ┆ 181           ┆ 3750        ┆ male   ┆ 2007 │
│ 2     ┆ Adelie    ┆ Torgersen ┆ 39.5           ┆ … ┆ 186           ┆ 3800        ┆ female ┆ 2007 │
│ 3     ┆ Adelie    ┆ Torgersen ┆ 40.3           ┆ … ┆ 195           ┆ 3250        ┆ female ┆ 2007 │
│ 4     ┆ Adelie    ┆ Torgersen ┆ NA             ┆ … ┆ NA            ┆ NA  

In [6]:
penguins = pl.read_csv("../data/penguins.csv", null_values="NA")
print(penguins)

shape: (344, 9)
┌───────┬───────────┬───────────┬────────────────┬───┬───────────────┬─────────────┬────────┬──────┐
│ rowid ┆ species   ┆ island    ┆ bill_length_mm ┆ … ┆ flipper_lengt ┆ body_mass_g ┆ sex    ┆ year │
│ ---   ┆ ---       ┆ ---       ┆ ---            ┆   ┆ h_mm          ┆ ---         ┆ ---    ┆ ---  │
│ i64   ┆ str       ┆ str       ┆ f64            ┆   ┆ ---           ┆ i64         ┆ str    ┆ i64  │
│       ┆           ┆           ┆                ┆   ┆ i64           ┆             ┆        ┆      │
╞═══════╪═══════════╪═══════════╪════════════════╪═══╪═══════════════╪═════════════╪════════╪══════╡
│ 1     ┆ Adelie    ┆ Torgersen ┆ 39.1           ┆ … ┆ 181           ┆ 3750        ┆ male   ┆ 2007 │
│ 2     ┆ Adelie    ┆ Torgersen ┆ 39.5           ┆ … ┆ 186           ┆ 3800        ┆ female ┆ 2007 │
│ 3     ┆ Adelie    ┆ Torgersen ┆ 40.3           ┆ … ┆ 195           ┆ 3250        ┆ female ┆ 2007 │
│ 4     ┆ Adelie    ┆ Torgersen ┆ null           ┆ … ┆ null          ┆ null

In [9]:
print(penguins.null_count().transpose(
    include_header=True, column_names=["null_count"]
))


shape: (9, 2)
┌───────────────────┬────────────┐
│ column            ┆ null_count │
│ ---               ┆ ---        │
│ str               ┆ u32        │
╞═══════════════════╪════════════╡
│ rowid             ┆ 0          │
│ species           ┆ 0          │
│ island            ┆ 0          │
│ bill_length_mm    ┆ 2          │
│ bill_depth_mm     ┆ 2          │
│ flipper_length_mm ┆ 2          │
│ body_mass_g       ┆ 2          │
│ sex               ┆ 11         │
│ year              ┆ 0          │
└───────────────────┴────────────┘


In [15]:
import chardet
def detect_encoding(filename: str) -> str:
    with open(filename, "rb") as f:
        raw_data = f.read()
        result = chardet.detect(raw_data)
        return result["encoding"]

print(detect_encoding("../data/directors.csv"))

EUC-JP


In [16]:
pl.read_csv("../data/directors.csv", encoding="EUC-JP")

name,born,country
str,i64,str
"""深作""",1930,"""日本"""
"""Verhoeven""",1938,"""オランダ"""
"""宮崎""",1942,"""日本"""
"""Tarantino""",1963,"""米国"""


In [19]:
songs = pl.read_excel("../data/top2000-2023.xlsx")
print(songs)

shape: (2_000, 4)
┌─────────┬─────────────────────────┬────────────────┬──────┐
│ positie ┆ titel                   ┆ artiest        ┆ jaar │
│ ---     ┆ ---                     ┆ ---            ┆ ---  │
│ i64     ┆ str                     ┆ str            ┆ i64  │
╞═════════╪═════════════════════════╪════════════════╪══════╡
│ 1       ┆ Bohemian Rhapsody       ┆ Queen          ┆ 1975 │
│ 2       ┆ Roller Coaster          ┆ Danny Vera     ┆ 2019 │
│ 3       ┆ Hotel California        ┆ Eagles         ┆ 1977 │
│ 4       ┆ Piano Man               ┆ Billy Joel     ┆ 1974 │
│ 5       ┆ Fix You                 ┆ Coldplay       ┆ 2005 │
│ …       ┆ …                       ┆ …              ┆ …    │
│ 1996    ┆ Charlie Brown           ┆ Coldplay       ┆ 2011 │
│ 1997    ┆ Beast Of Burden         ┆ Bette Midler   ┆ 1984 │
│ 1998    ┆ It Was A Very Good Year ┆ Frank Sinatra  ┆ 1968 │
│ 1999    ┆ Hou Van Mij             ┆ 3JS            ┆ 2008 │
│ 2000    ┆ Drivers License         ┆ Olivia Rodrigo

In [21]:
print(pl.read_csv("../data/stock/nvda/201?.csv"))

shape: (2_516, 8)
┌────────┬────────────┬───────────┬───────────┬───────────┬───────────┬───────────┬──────────┐
│ symbol ┆ date       ┆ open      ┆ high      ┆ low       ┆ close     ┆ adj close ┆ volume   │
│ ---    ┆ ---        ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│ str    ┆ str        ┆ f64       ┆ f64       ┆ f64       ┆ f64       ┆ f64       ┆ i64      │
╞════════╪════════════╪═══════════╪═══════════╪═══════════╪═══════════╪═══════════╪══════════╡
│ NVDA   ┆ 2010-01-04 ┆ 4.6275    ┆ 4.655     ┆ 4.5275    ┆ 4.6225    ┆ 4.240429  ┆ 80020400 │
│ NVDA   ┆ 2010-01-05 ┆ 4.605     ┆ 4.74      ┆ 4.605     ┆ 4.69      ┆ 4.30235   ┆ 72864800 │
│ NVDA   ┆ 2010-01-06 ┆ 4.6875    ┆ 4.73      ┆ 4.6425    ┆ 4.72      ┆ 4.32987   ┆ 64916800 │
│ NVDA   ┆ 2010-01-07 ┆ 4.695     ┆ 4.715     ┆ 4.5925    ┆ 4.6275    ┆ 4.245015  ┆ 54779200 │
│ NVDA   ┆ 2010-01-08 ┆ 4.59      ┆ 4.67      ┆ 4.5625    ┆ 4.6375    ┆ 4.254189  ┆ 47816800 │
│ …      ┆ …          ┆ …       

In [22]:
all_stocks = pl.read_csv("../data/stock/**/*.csv")
print(all_stocks)

shape: (18_476, 8)
┌────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬──────────┐
│ symbol ┆ date       ┆ open       ┆ high       ┆ low        ┆ close      ┆ adj close  ┆ volume   │
│ ---    ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---        ┆ ---      │
│ str    ┆ str        ┆ f64        ┆ f64        ┆ f64        ┆ f64        ┆ f64        ┆ i64      │
╞════════╪════════════╪════════════╪════════════╪════════════╪════════════╪════════════╪══════════╡
│ ASML   ┆ 1999-01-04 ┆ 11.765625  ┆ 12.28125   ┆ 11.765625  ┆ 12.140625  ┆ 7.522523   ┆ 1801867  │
│ ASML   ┆ 1999-01-05 ┆ 11.859375  ┆ 14.25      ┆ 11.71875   ┆ 13.96875   ┆ 8.655257   ┆ 8241600  │
│ ASML   ┆ 1999-01-06 ┆ 14.25      ┆ 17.601563  ┆ 14.203125  ┆ 16.875     ┆ 10.456018  ┆ 16400267 │
│ ASML   ┆ 1999-01-07 ┆ 14.742188  ┆ 17.8125    ┆ 14.53125   ┆ 16.851563  ┆ 10.441495  ┆ 17722133 │
│ ASML   ┆ 1999-01-08 ┆ 16.078125  ┆ 16.289063  ┆ 15.023438  ┆ 15.796875  ┆ 9.787

In [24]:
import calendar

filenames = [
    f"../data/stock/asml/{year}.csv"
    for year in range(1999, 2024)
    if calendar.isleap(year)
]

filenames

['../data/stock/asml/2000.csv',
 '../data/stock/asml/2004.csv',
 '../data/stock/asml/2008.csv',
 '../data/stock/asml/2012.csv',
 '../data/stock/asml/2016.csv',
 '../data/stock/asml/2020.csv']

In [29]:
start = time.time()
trips = pl.read_parquet("../data/taxi/yellow_tripdata_*.parquet")
print(trips)
print(f"Time: {time.time() - start:.2f}s")

shape: (39_656_098, 19)
┌──────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ VendorID ┆ tpep_pick ┆ tpep_drop ┆ passenger ┆ … ┆ improveme ┆ total_amo ┆ congestio ┆ airport_f │
│ ---      ┆ up_dateti ┆ off_datet ┆ _count    ┆   ┆ nt_surcha ┆ unt       ┆ n_surchar ┆ ee        │
│ i64      ┆ me        ┆ ime       ┆ ---       ┆   ┆ rge       ┆ ---       ┆ ge        ┆ ---       │
│          ┆ ---       ┆ ---       ┆ f64       ┆   ┆ ---       ┆ f64       ┆ ---       ┆ f64       │
│          ┆ datetime[ ┆ datetime[ ┆           ┆   ┆ f64       ┆           ┆ f64       ┆           │
│          ┆ ns]       ┆ ns]       ┆           ┆   ┆           ┆           ┆           ┆           │
╞══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ 1        ┆ 2022-01-0 ┆ 2022-01-0 ┆ 2.0       ┆ … ┆ 0.3       ┆ 21.95     ┆ 2.5       ┆ 0.0       │
│          ┆ 1         ┆ 1         ┆           ┆   ┆           ┆   

In [31]:
pokedex = pl.read_json("../data/pokedex.json")
pokedex

pokemon
list[struct[17]]
"[{1,""001"",""Bulbasaur"",""http://www.serebii.net/pokemongo/pokemon/001.png"",[""Grass"", ""Poison""],""0.71 m"",""6.9 kg"",""Bulbasaur Candy"",25,""2 km"",0.69,69.0,""20:00"",[1.58],[""Fire"", ""Ice"", … ""Psychic""],[{""002"",""Ivysaur""}, {""003"",""Venusaur""}],null}, {2,""002"",""Ivysaur"",""http://www.serebii.net/pokemongo/pokemon/002.png"",[""Grass"", ""Poison""],""0.99 m"",""13.0 kg"",""Bulbasaur Candy"",100,""Not in Eggs"",0.042,4.2,""07:00"",[1.2, 1.6],[""Fire"", ""Ice"", … ""Psychic""],[{""003"",""Venusaur""}],[{""001"",""Bulbasaur""}]}, … {151,""151"",""Mew"",""http://www.serebii.net/pokemongo/pokemon/151.png"",[""Psychic""],""0.41 m"",""4.0 kg"",""None"",null,""Not in Eggs"",0.0,0.0,""N/A"",null,[""Bug"", ""Ghost"", ""Dark""],null,null}]"


In [32]:
(
    pokedex.explode("pokemon")
    .unnest("pokemon")
    .select("id", "name", "type", "height", "weight")
)

id,name,type,height,weight
i64,str,list[str],str,str
1,"""Bulbasaur""","[""Grass"", ""Poison""]","""0.71 m""","""6.9 kg"""
2,"""Ivysaur""","[""Grass"", ""Poison""]","""0.99 m""","""13.0 kg"""
3,"""Venusaur""","[""Grass"", ""Poison""]","""2.01 m""","""100.0 kg"""
4,"""Charmander""","[""Fire""]","""0.61 m""","""8.5 kg"""
5,"""Charmeleon""","[""Fire""]","""1.09 m""","""19.0 kg"""
…,…,…,…,…
147,"""Dratini""","[""Dragon""]","""1.80 m""","""3.3 kg"""
148,"""Dragonair""","[""Dragon""]","""3.99 m""","""16.5 kg"""
149,"""Dragonite""","[""Dragon"", ""Flying""]","""2.21 m""","""210.0 kg"""


In [34]:
from json import loads
from pprint import pprint

with open("../data/wikimedia.ndjson") as f:
    pprint(loads(f.readline()))

{'$schema': '/mediawiki/recentchange/1.0.0',
 'bot': False,
 'comment': '/* League champions, runners-up and play-off finalists */',
 'id': 1659529639,
 'length': {'new': 91166, 'old': 91108},
 'meta': {'domain': 'en.wikipedia.org',
          'dt': '2023-07-29T07:51:39Z',
          'id': '0416300b-980c-45bb-b0a2-c9d7a9e2b7eb',
          'offset': 4820784717,
          'partition': 0,
          'request_id': 'ea0541fb-4e72-4fc3-82f0-6c26651b2043',
          'stream': 'mediawiki.recentchange',
          'topic': 'eqiad.mediawiki.recentchange',
          'uri': 'https://en.wikipedia.org/wiki/EFL_Championship'},
 'minor': False,
 'namespace': 0,
 'notify_url': 'https://en.wikipedia.org/w/index.php?diff=1167689309&oldid=1166824248',
 'parsedcomment': '<span dir="auto"><span class="autocomment"><a '
                  'href="/wiki/EFL_Championship#League_champions,_runners-up_and_play-off_finalists" '
                  'title="EFL Championship">в†’вЂЋLeague champions, runners-up '
           

In [35]:
wikimedia = pl.read_ndjson("../data/wikimedia.ndjson")
print(wikimedia)

shape: (100, 20)
┌─────────────┬────────────┬────────────┬──────┬───┬────────────┬────────────┬────────┬────────────┐
│ $schema     ┆ meta       ┆ id         ┆ type ┆ … ┆ server_nam ┆ server_scr ┆ wiki   ┆ parsedcomm │
│ ---         ┆ ---        ┆ ---        ┆ ---  ┆   ┆ e          ┆ ipt_path   ┆ ---    ┆ ent        │
│ str         ┆ struct[9]  ┆ i64        ┆ str  ┆   ┆ ---        ┆ ---        ┆ str    ┆ ---        │
│             ┆            ┆            ┆      ┆   ┆ str        ┆ str        ┆        ┆ str        │
╞═════════════╪════════════╪════════════╪══════╪═══╪════════════╪════════════╪════════╪════════════╡
│ /mediawiki/ ┆ {"https:// ┆ 1659529639 ┆ edit ┆ … ┆ en.wikiped ┆ /w         ┆ enwiki ┆ <span dir= │
│ recentchang ┆ en.wikiped ┆            ┆      ┆   ┆ ia.org     ┆            ┆        ┆ "auto"><sp │
│ e/1.0.0     ┆ ia.org/wik ┆            ┆      ┆   ┆            ┆            ┆        ┆ an         │
│             ┆ …          ┆            ┆      ┆   ┆            ┆         